In [42]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import sqlite3
from scipy.optimize import curve_fit
from matplotlib.backends import backend_pdf

In [43]:
# SUPPORT FUNCTIONS TO CONVERT CHANNEL ID TO BOARD/PMT ID
# using hardware database inside icarus_data

def readSqlitedb(database="/cvmfs/icarus.opensciencegrid.org/products/icarus/icarus_data/v09_79_01/icarus_data/database/ChannelMapICARUS_20230829.db", table="pmt_placements_29aug2023"):

    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(database)
    df = pd.read_sql_query("SELECT * from {}".format(table), con)

    con.close()

    return df

def get_channel_id(db, board, digitizer_channel):
    
    ch = db.loc[(db['digitizer_label']==board) & (db['digitizer_ch_number']==digitizer_channel), ['channel_id']]
    # print(board, ",", digitizer_channel, "-->", ch.values[0][0])

    return ch.values[0][0]

def get_PMT_id(db, channels):
     
    if np.isscalar(channels):
        pmt_id = db[db.channel_id==channels].pmt_id.values[0]
        return pmt_id
    else:
        pmt_ids = [ db[db.channel_id==ch].pmt_id.values[0] for ch in channels ] 
        return pmt_ids
    
def get_channel_board(db, ch):
    
    board = db.loc[db['channel_id']==ch].digitizer_label.values[0]
    return board

def get_board_label(filename):
    
    head, tail = os.path.split(filename)
    buf = tail.removesuffix(".fcl").removeprefix("icaruspmt")
    geo = buf[0:2]
    pos = buf[2:-2]
    slot = buf[-2:]
  
    if slot == "01":
        slot = "A"
    elif slot == "02":
        slot = "B"
    elif slot == "03":
        slot = "C"

    board_label = geo.upper() + "-" + pos.upper() + "-" + slot
    
    return board_label

In [44]:
### GET REAL CURRENT BASELINES (AVOIDING MAPPING ISSUES- DIO BESTIA!!!!)
def readFile( path ):
    
    channel_id = []
    nominal_offset = []
    baselines = []
    
    pmt_components = [ filename for filename in os.listdir(path)  if "icaruspmt" in filename]
    #print(pmt_components)
    for filename in pmt_components:
        
        readfile = open( path+"/"+filename, 'r' )
        board_label = get_board_label(filename)
        map = readSqlitedb()
        
        for line in readfile:
            line = line.strip()
            if "channelPedestal" in line:
                argline = line.split(".")[-1]
                buff = argline.split(":")
                digitizer_channel = int(buff[0].split("channelPedestal")[-1]) #these go from 0 to 15
                dc_offset_old = int(buff[1])
                
                if (digitizer_channel != 15): #last digitizer channel has no pmt
                    channel_id.append(get_channel_id(map, board_label, digitizer_channel)) #in db, expected 0 to 15
                    nominal_offset.append(dc_offset_old)
                    baselines.append(14747)
        
        readfile.close()

    return channel_id, nominal_offset, baselines


path = "../pmt-thresholds-baselines/basedir/PMTlaser"
channel_ids, nominal, baselines = readFile(path)
channel_ids = np.array(channel_ids)
nominal = np.array(nominal)
baselines = np.array(baselines)
print(channel_ids, nominal)

[ 14  11   4   1   7   9  12  10   2   0   8   5   3   6  13 134 131 124
 121 127 129 130 132 120 122 125 128 123 126 133 284 281 274 271 277 279
 280 282 270 272 275 278 273 276 283 164 161 154 151 157 159 160 162 150
 152 155 158 153 156 163 314 311 304 301 307 309 310 312 300 302 305 308
 303 306 313 114 111 117 119 107 109 110 112 115 118 105 108 113 116 106
 104 101  94  91  97  99 100 102  90  92  95  98  93  96 103  84  81  87
  89  77  79  82  80  88  85  78  75  83  86  76 234 231 237 239 227 229
 232 230 238 235 228 225 233 236 226  74  71  64  61  67  69  72  70  62
  60  68  65  63  66  73 324 321 327 329 317 319 320 322 325 328 315 318
 323 326 316  24  21  27  29  17  19  22  20  28  25  18  15  23  26  16
 264 261 267 269 257 259 262 260 268 265 258 255 263 266 256 254 251 244
 241 247 249 252 250 242 240 248 245 243 246 253 144 141 147 149 137 139
 140 142 145 148 135 138 143 146 136 294 291 297 299 287 289 290 292 295
 298 285 288 293 296 286 354 351 357 359 347 349 35

In [45]:
## GET CURRENT BASELINES
#file = "calibrations/DC_offsets_for_14747_4th_3Mar2023.csv"
#df_nominal = pd.read_csv(file,sep=",")
#df_nominal = df_nominal[["channel_id","baseline","DC_offset"]]
#df_nominal.head()

In [46]:
m5 = nominal - 0.05*nominal
p5 = nominal + 0.05*nominal
m10 = nominal - 0.10*nominal
p10 = nominal + 0.10*nominal
m20 = nominal - 0.20*nominal
p20 = nominal + 0.20*nominal

In [47]:
#print(m20, p20)
d = {
    "nom": nominal,
    "m5": m5,
    "p5": p5,
    "m10": m10,
    "p10": p10,
    "m20": m20,
    "p20": p20, 
}

In [48]:
for label, vector in d.items():
    path = "baseline_scans/DC_offsets_{}.csv".format(label)
    ofile = open(path,"w")
    header = "channel_id,baseline,DC_offset\n"
    ofile.write(header)

    for i, channel in enumerate(channel_ids):
        out = str(channel) + "," + str(baselines[i]) + ","
        out += str(int(vector[i]))+"\n"
        ofile.write(out)
    ofile.close()